In [10]:
FILE_NAME = '../03_snapshot_proposals/1687557600_snapshot_proposals.csv'

RECOVERY_DIR_NAME = '1687946703'
# RECOVERY_DIR_NAME = None
RECOVERY_MODE = RECOVERY_DIR_NAME is not None


In [11]:
import pandas as pd
proposals_df = pd.read_csv(FILE_NAME)
# rename daoId to deepdaoDaoId
proposals_df = proposals_df.rename(columns={'daoId': 'deepdaoDaoId'})
proposals_df.head()


,snapshotSpaceId,deepdaoDaoId,snapshotProposalId,ipfs,author,created,network,symbol,type,strategies,...,state,link,app,scores,scores_by_strategy,scores_state,scores_total,scores_updated,votes,flagged
0,01meta.eth,01meta.eth,0xca5d7498f61672cd84db8e2cc1cf54a7b7abbc8df0fc...,bafkreib2itqhuonga5qrjibendaz4sdgluas7a5rweuuv...,0x593E91E0Ab8eDa2d92ea0879cD95C641f877Ce03,2023-06-23 13:27:59,137.0,Matic,single-choice,"[{'network': '137', 'params': {'symbol': 'mati...",...,active,https://snapshot.org/#/01meta.eth/proposal/0xc...,snapshot,"[1.927831279399148, 0.2]","[[1.927831279399148], [0.2]]",pending,2.127831,2023-06-23 13:28:54,2.0,False
1,01meta.eth,01meta.eth,0x77544bf8ce89c580bdf5b7c0522e8570832c86e55718...,bafkreic26ur2p42fr52dkcixeshmjo2hvvl2pjb5xrlcm...,0x891cE83F01EB7fc506149068C188fDC599022345,2023-06-16 14:01:47,137.0,Matic,single-choice,"[{'network': '137', 'params': {'symbol': 'mati...",...,active,https://snapshot.org/#/01meta.eth/proposal/0x7...,snapshot,"[0.6004409257825702, 0]","[[0.6004409257825702], [0]]",pending,0.600441,2023-06-23 17:16:56,2.0,False
2,01meta.eth,01meta.eth,0xe493e04e0b2750c2f7601ccd2a285a024e67f39527f5...,QmWZha1jM3HroQEBtkKAbEF8jKiX6aytsq69dPgHWVV4xC,0x5CC249A44947C00134c20508949E79916ABdff47,2023-06-16 07:04:27,137.0,Matic,single-choice,"[{'network': '137', 'params': {'symbol': 'mati...",...,active,https://snapshot.org/#/01meta.eth/proposal/0xe...,snapshot,"[0, 0.2]","[[0], [0.2]]",pending,0.200000,2023-06-16 07:06:12,1.0,False
3,01meta.eth,01meta.eth,0xb90f3fd67b646b0c91c3cbaf17571fea30e20ed8ae12...,QmVDrSy36MQYUU6iQohS6jD8k3L7tgxCTx5Y7EeAybBrtH,0x0616072d6937245C9186B63adC4e746F576b777c,2023-06-16 06:45:04,137.0,Matic,single-choice,"[{'network': '137', 'params': {'symbol': 'mati...",...,active,https://snapshot.org/#/01meta.eth/proposal/0xb...,snapshot,"[0, 0.09572972959688068]","[[0], [0.09572972959688068]]",pending,0.095730,2023-06-16 06:47:14,1.0,False
4,01meta.eth,01meta.eth,0xd8a948e7c1b0c2a6e0c9e9ef3e613dbfafbe428a06b2...,bafkreiby7atcls73fjfmkms36xrmooyrjgdt7br3nyfpi...,0xC71A411E57baf8b4aC8eCf75C01972cf1E9C464e,2023-06-16 06:27:52,137.0,Matic,single-choice,"[{'network': '137', 'params': {'symbol': 'mati...",...,active,https://snapshot.org/#/01meta.eth/proposal/0xd...,snapshot,"[0, 0.01]","[[0], [0.01]]",pending,0.010000,2023-06-16 06:29:29,1.0,False


In [12]:
# only keep the id and title columns
proposals_df = proposals_df[['deepdaoDaoId', 'snapshotProposalId', 'snapshotSpaceId', 'title']]
proposals_df.head()


,deepdaoDaoId,snapshotProposalId,snapshotSpaceId,title
0,01meta.eth,0xca5d7498f61672cd84db8e2cc1cf54a7b7abbc8df0fc...,01meta.eth,What's nice fam?
1,01meta.eth,0x77544bf8ce89c580bdf5b7c0522e8570832c86e55718...,01meta.eth,I am so HIGH - GIB ME DROP
2,01meta.eth,0xe493e04e0b2750c2f7601ccd2a285a024e67f39527f5...,01meta.eth,double trouble
3,01meta.eth,0xb90f3fd67b646b0c91c3cbaf17571fea30e20ed8ae12...,01meta.eth,Willi billy
4,01meta.eth,0xd8a948e7c1b0c2a6e0c9e9ef3e613dbfafbe428a06b2...,01meta.eth,good time passed


In [13]:
proposals_df.shape


(15182, 4)

In [14]:
def generate_query(proposalId, first, skip):
    base = """
        query Votes {
          votes (
            where: {
              proposal: "%s"
            }
            first: %s
            skip: %s
            orderBy: "created",
            orderDirection: desc
          ) {
            id
            ipfs
            voter
            created
            choice
            metadata
            reason
            app
            vp
            vp_by_strategy
            vp_state
          }
        }
    """
    return base % (proposalId, first, skip)


In [15]:
import requests
import json
import pandas as pd

import random
import time

URL = "https://hub.snapshot.org/graphql"


def get_all_votes(proposalId):
    all_votes = []

    has_next_page = True
    skip = 0
    while has_next_page:
        print('.')

        query = generate_query(proposalId, 1000, skip)
        r = requests.post(URL, json={'query': query})
        try:
            data = json.loads(r.text)
        except:
            print(r.text)
            raise Exception('failed to parse json')

        votes = data['data']['votes']
        if len(votes) == 0:
            has_next_page = False
            break
        all_votes += votes

        if len(votes) < 1000:
            has_next_page = False
            break
        wait_time = random.randint(1, 10)
        print(f'waiting {wait_time} seconds')
        time.sleep(wait_time)

        skip += 1000


    all_votes_df = pd.DataFrame(all_votes)

    # add proposalId to the df
    all_votes_df['proposalId'] = proposalId
    return all_votes_df



In [16]:
# get the current epoch and make a directory
import time
import datetime

if not RECOVERY_MODE:
    now = datetime.datetime.now()
    epoch = int(time.mktime(now.timetuple()))
    output_dir_name = str(epoch)
else:
    output_dir_name = RECOVERY_DIR_NAME


In [17]:
# make the output dir
import os
if not os.path.exists(f'{output_dir_name}'):
    os.makedirs(f'{output_dir_name}')

# make a directory for each of the deepdaoDaoIds
for deepdaoDaoId in proposals_df['deepdaoDaoId'].unique():
    subdir = f'{output_dir_name}/{deepdaoDaoId}'
    if not os.path.exists(subdir):
        os.makedirs(subdir)



In [18]:
from tqdm import tqdm

print('using directory', output_dir_name)

for row in tqdm(proposals_df.itertuples(), total=proposals_df.shape[0]):
    proposal_id = row.snapshotProposalId  # this is the key to query the api
    deepdao_dao_id = row.deepdaoDaoId

    if os.path.exists(f'{output_dir_name}/{deepdao_dao_id}/{proposal_id}.csv'):
        print(f'{proposal_id} csv already exists')
        continue
    votes = get_all_votes(proposal_id)

    # save in the format <epoch>/<deepdaoDaoId>/<proposalId>.csv
    proposal_votes_file_name = f'{output_dir_name}/{deepdao_dao_id}/{proposal_id}.csv'
    print(f'saving to {proposal_votes_file_name}')
    votes.to_csv(proposal_votes_file_name, index=False)

    # wait between each proposal
    wait_time = random.randint(1, 10)
    print(f'waiting {wait_time} seconds')
    time.sleep(wait_time)


  0%|          | 0/15182 [00:00<?, ?it/s]

0xca5d7498f61672cd84db8e2cc1cf54a7b7abbc8df0fc7c2d699e510587f9133d csv already exists
0x77544bf8ce89c580bdf5b7c0522e8570832c86e557189185c133b917289ba61b csv already exists
0xe493e04e0b2750c2f7601ccd2a285a024e67f39527f5d6a5e0abe16dc9af21c2 csv already exists
.
saving to 1687946703/01meta.eth/0xb90f3fd67b646b0c91c3cbaf17571fea30e20ed8ae129b8503de1903fe4b9994.csv
waiting 3 seconds


  0%|          | 4/15182 [00:03<3:31:52,  1.19it/s]

.
saving to 1687946703/01meta.eth/0xd8a948e7c1b0c2a6e0c9e9ef3e613dbfafbe428a06b2378e1c08089b9e292a66.csv
waiting 8 seconds


  0%|          | 5/15182 [00:11<11:46:47,  2.79s/it]

.
saving to 1687946703/01meta.eth/0xf7096d889a32ede1d43df25155c3554b694dc4c840346103cb53f351b487f2fc.csv
waiting 9 seconds


  0%|          | 5/15182 [00:15<13:03:16,  3.10s/it]


KeyboardInterrupt: 